In [2]:
# Loading in data

import pandas as pd

df = pd.read_csv('data/Colorado River Basin Water Conflict Table.csv')

In [3]:
df

,Event,Search Source,Newspaper,Article Title,Duplicate,Report Date,Report Year,Event Date,Event Day,Event Month,...,Article Text Search - water rights,Article Text Search - intergovernmental,Article Text Search - water transfers,Article Text Search - navigation,Article Text Search - fish,Article Text Search - invasive,Article Text Search - diversion,Article Text Search - water diversion,Article Text Search - instream,Article Text Search - aquatic
0,1,USGS1-50.docx,The Durango Herald (Colorado),Tribes assert water rights on Colorado River B...,False,7-Apr-22,2022.0,NaN,NaN,4.0,...,17,0,0,0,0,0,0,0,0,0
1,2,USGS1-50.docx,"Journal, The (Cortez, Dolores, Mancos, CO)",Native American tribes assert water rights on ...,False,7-Apr-22,2022.0,NaN,NaN,4.0,...,17,0,0,0,0,0,0,0,0,0
2,3,USGS1-50.docx,The Salt Lake Tribune,'Very positive change.' New Utah law will be a...,False,17-Mar-22,2022.0,NaN,NaN,3.0,...,12,0,0,0,1,0,0,0,12,1
3,4,USGS1-50.docx,Casa Grande Dispatch (AZ),Legislation would let an Arizona tribe lease C...,False,11-Dec-21,2021.0,NaN,NaN,12.0,...,6,0,0,0,0,0,0,0,0,0
4,5,USGS1-50.docx,The Aspen Times (Colorado),Historically excluded from Colorado River poli...,False,19-Dec-21,2021.0,NaN,NaN,11.0,...,18,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,264,USGS301-350.docx,The Durango Herald (Colorado),Water officials consider action for worst case...,False,9-Jun-09,2019.0,6,NaN,6.0,...,3,0,0,0,0,0,0,0,0,0
264,265,USGS301-350.docx,"Rio Blanco Herald Times (Meeker, Colorado)",Rangely hosts Colorado River District event,False,21-Apr-22,2022.0,4/13/2022,13.0,4.0,...,3,0,0,0,0,1,1,0,0,0
265,266,USGS301-350.docx,Casa Grande Dispatch (AZ),California water district lawsuit threatens dr...,False,18-Apr-19,2019.0,4,NaN,4.0,...,2,0,0,0,0,0,0,0,0,0
266,267,USGS301-350.docx,The Salt Lake Tribune,Scientists want to flush water past Glen Canyo...,False,13-Dec-07,2007.0,12,NaN,12.0,...,0,0,0,0,0,2,0,0,0,0
